In [32]:
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, SeparableConv1D, Dropout, Dense, Flatten

In [14]:
file_data = pd.read_csv('./\Breath_cycles/file_data.csv')
file_data

,fname,start,end,crack_wheez,fname_cycle
0,101_1b1_Al_sc_Meditron,0.036,0.579,0,101_1b1_Al_sc_Meditron_0.wav
1,101_1b1_Al_sc_Meditron,0.579,2.450,0,101_1b1_Al_sc_Meditron_1.wav
2,101_1b1_Al_sc_Meditron,2.450,3.893,0,101_1b1_Al_sc_Meditron_2.wav
3,101_1b1_Al_sc_Meditron,3.893,5.793,0,101_1b1_Al_sc_Meditron_3.wav
4,101_1b1_Al_sc_Meditron,5.793,7.521,0,101_1b1_Al_sc_Meditron_4.wav
...,...,...,...,...,...
6893,226_1b1_Pl_sc_LittC2SE,11.721,13.693,1,226_1b1_Pl_sc_LittC2SE_6.wav
6894,226_1b1_Pl_sc_LittC2SE,13.693,15.536,0,226_1b1_Pl_sc_LittC2SE_7.wav
6895,226_1b1_Pl_sc_LittC2SE,15.536,17.493,0,226_1b1_Pl_sc_LittC2SE_8.wav
6896,226_1b1_Pl_sc_LittC2SE,17.493,19.436,1,226_1b1_Pl_sc_LittC2SE_9.wav


In [15]:
train_df, test_df = train_test_split(file_data, test_size=0.1, random_state=42, stratify = file_data['crack_wheez'])
train_df

,fname,start,end,crack_wheez,fname_cycle
2235,145_3b4_Pl_mc_AKGC417L,15.7350,19.1400,0,145_3b4_Pl_mc_AKGC417L_4.wav
3029,157_1b1_Al_sc_Meditron,19.3580,23.0300,0,157_1b1_Al_sc_Meditron_3.wav
952,124_1b1_Lr_sc_Litt3200,4.4274,5.4465,0,124_1b1_Lr_sc_Litt3200_3.wav
5611,201_1b1_Al_sc_Meditron,2.0070,5.1220,1,201_1b1_Al_sc_Meditron_1.wav
4691,178_2b2_Ar_mc_AKGC417L,8.3870,10.5180,0,178_2b2_Ar_mc_AKGC417L_4.wav
...,...,...,...,...,...
4285,175_1b1_Al_sc_Litt3200,3.5720,6.6693,3,175_1b1_Al_sc_Litt3200_1.wav
6518,217_1b1_Tc_sc_Meditron,11.1070,13.2930,0,217_1b1_Tc_sc_Meditron_6.wav
1020,130_1p2_Ar_mc_AKGC417L,0.6970,2.4230,1,130_1p2_Ar_mc_AKGC417L_0.wav
1598,133_2p3_Ar_mc_AKGC417L,9.9460,12.5420,0,133_2p3_Ar_mc_AKGC417L_4.wav


In [16]:
#Reset index
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [17]:
def audio_features(filename): 
    sound, sample_rate = librosa.load('./\Breath_cycles/'+filename, sr=None)
    stft = np.abs(librosa.stft(sound))   
    mfccs = np.mean(librosa.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40),axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate),axis=1)
    mel = np.mean(librosa.feature.melspectrogram(sound, sr=sample_rate),axis=1)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate, fmin = 30),axis=1)
    #tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound), sr=sample_rate, chroma=librosa.feature.chroma_cqt(y=sound, sr=sample_rate, fmin=30, bins_per_octave =30)),axis=1)    
    #concat = np.concatenate((mfccs,chroma,mel,contrast,tonnetz))
    concat = np.concatenate((mfccs,chroma,mel,contrast))
    return concat

In [18]:
def build_feat(df):
    X = []
    Y = []    
    for i in tqdm(range(len(df))):
        file = df.iloc[i].fname_cycle
        label = df.iloc[i].crack_wheez 
        X_sample = audio_features(file)
        X.append(X_sample)
        Y.append(label)
    X, Y = np.array(X), np.array(Y)
    return X, Y

In [21]:
X_, y_ = build_feat(train_df)

  1%|█▌                                                                                                                                                                                          | 50/6208 [00:00<01:30, 67.89it/s]c:\users\parva\appdata\local\programs\python\python38\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1772
  warnings.warn(
  2%|███▏                                                                                                                                                                                       | 107/6208 [00:01<01:30, 67.72it/s]c:\users\parva\appdata\local\programs\python\python38\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=2040
  warnings.warn(
  4%|██████▋                                                                                                                                                              

 69%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                         | 4302/6208 [01:03<00:27, 69.87it/s]c:\users\parva\appdata\local\programs\python\python38\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1808
  warnings.warn(
 74%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                 | 4570/6208 [01:07<00:22, 71.58it/s]c:\users\parva\appdata\local\programs\python\python38\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1888
  warnings.warn(
 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                        

In [65]:
y_

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [58]:

X_.shape

(6208, 187)

In [19]:
X_test, y_test = build_feat(test_df)

 70%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                       | 486/690 [00:07<00:02, 68.19it/s]c:\users\parva\appdata\local\programs\python\python38\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1315
  warnings.warn(
 78%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                          | 536/690 [00:07<00:02, 67.53it/s]c:\users\parva\appdata\local\programs\python\python38\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1468
  warnings.warn(
 84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████       

In [23]:
X_tr, X_val, y_tr, y_val = train_test_split(X_, y_, test_size=0.1, random_state=42)

In [24]:
X_test.shape


(690, 187)

In [46]:
X_tr.shape[1]


187

In [47]:
y_tr

array([1, 0, 2, ..., 1, 2, 2], dtype=int64)

In [40]:
X_val.shape

(621, 187)

In [41]:
y_val.shape

(621,)

In [53]:
y_tr = np.reshape(y_tr, (y_tr.shape[0], 1))
X_tr = np.reshape(X_tr, (X_tr.shape[0], X_tr.shape[1], 1))
y_val = np.reshape(y_val, (y_val.shape[0], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],  1))

In [54]:
model = Sequential()
model.add(Conv1D(64, kernel_size=5, activation='relu', input_shape=(187,1)))

model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(2)) 

model.add(SeparableConv1D(256, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(2)) 

model.add(SeparableConv1D(256, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(2)) 

model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(512, activation='relu'))   
model.add(Dense(1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_tr, y_tr, validation_data=(X_val, y_val), epochs=70, batch_size=200, verbose=1)

Epoch 1/70
28/28 [==============================] - 8s 280ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 2/70
28/28 [==============================] - 8s 284ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 3/70
28/28 [==============================] - 8s 285ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 4/70
28/28 [==============================] - 9s 322ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 5/70
28/28 [==============================] - 9s 322ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 6/70
28/28 [==============================] - 9s 315ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 7/70
28/28 [==============================] - 8s 297ms/step - loss: 0.0000e+00 - accuracy: 0.2

28/28 [==============================] - 8s 289ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 56/70
28/28 [==============================] - 8s 293ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 57/70
28/28 [==============================] - 8s 291ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 58/70
28/28 [==============================] - 8s 291ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 59/70
28/28 [==============================] - 8s 292ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 60/70
28/28 [==============================] - 8s 290ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2834
Epoch 61/70
28/28 [==============================] - 8s 289ms/step - loss: 0.0000e+00 - accuracy: 0.2688 -